In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
import os

if IN_COLAB:
    ! pip3 install mne
    ! pip3 install yadisk

    import yadisk

    if not os.path.exists('dataset.zip'):
        url = 'https://disk.yandex.ru/d/hItiy8VK-lk0gQ' 
        y = yadisk.YaDisk()
        y.download_public(url, 'dataset.zip')

        ! unzip dataset.zip

    if not os.path.exists('eegproject'):
        ! unzip eegproject.zip

In [ ]:
import torch
import random
import numpy as np

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_random_seed(42)

In [ ]:
from eegproject.data.sequential_dataset import SequentialEEGDataset, get_short_sequence_dataset, iterate_batches
from eegproject.data.transforms import scale

In [ ]:
train_dataset_ = SequentialEEGDataset(split='train', preprocessed_path='eeg_dataset/train_eeg_dataset.pt', transform=scale)
test_dataset_ = SequentialEEGDataset(split='test', preprocessed_path='eeg_dataset/test_eeg_dataset.pt', transform=scale)

In [ ]:
import math

def iterate_batch_multiple_seq_minibatches(inputs, targets, batch_size, seq_length):
    assert len(inputs) == len(targets)
    n_inputs = len(inputs)

    input_sample_shape = inputs[0].shape[1:]
    target_sample_shape = targets[0].shape[1:]
    
    seq_idx = np.arange(n_inputs)
    np.random.shuffle(seq_idx)
    
    # Compute the number of maximum loops
    n_loops = int(math.ceil(len(seq_idx) / batch_size))

    # For each batch of subjects (size=batch_size)
    for l in range(n_loops):
        start_idx = l*batch_size
        end_idx = (l+1)*batch_size
        seq_inputs = np.asarray(inputs)[seq_idx[start_idx:end_idx]]
        seq_targets = np.asarray(targets)[seq_idx[start_idx:end_idx]]

        max_skips = 5
        for s_idx in range(len(seq_inputs)):
            n_skips = np.random.randint(max_skips)
            seq_inputs[s_idx] = seq_inputs[s_idx][n_skips:]
            seq_targets[s_idx] = seq_targets[s_idx][n_skips:]

        # Determine the maximum number of batch sequences
        n_max_seq_inputs = -1
        for s_idx, s in enumerate(seq_inputs):
            if len(s) > n_max_seq_inputs:
                n_max_seq_inputs = len(s)

        n_batch_seqs = int(math.ceil(n_max_seq_inputs / seq_length))

        # For each batch sequence (size=seq_length)
        for b in range(n_batch_seqs):
            start_loop = True if b == 0 else False
            start_idx = b*seq_length
            end_idx = (b+1)*seq_length
            batch_inputs = np.zeros((batch_size, seq_length) + input_sample_shape, dtype=np.float32)
            batch_targets = np.ones((batch_size, seq_length) + target_sample_shape, dtype=int) * -1
            batch_seq_len = np.zeros(batch_size, dtype=int)
            # For each subject
            for s_idx, s in enumerate(zip(seq_inputs, seq_targets)):
                # (seq_len, sample_shape)
                each_seq_inputs = s[0][start_idx:end_idx]
                each_seq_targets = s[1][start_idx:end_idx]
                batch_inputs[s_idx, :len(each_seq_inputs)] = each_seq_inputs
                batch_targets[s_idx, :len(each_seq_targets)] = each_seq_targets
                batch_seq_len[s_idx] = len(each_seq_inputs)
            batch_x = torch.tensor(batch_inputs).float() # .reshape((-1,) + input_sample_shape)
            batch_y = torch.tensor(batch_targets) # .reshape((-1,) + target_sample_shape)
            yield batch_x, batch_y, start_loop

In [ ]:
import matplotlib.pyplot as plt

cnt = 0

for x, y, sl in iterate_batch_multiple_seq_minibatches(train_dataset_.X, train_dataset_.y, 32, 10):
    print(x.shape)
    break
    
cnt

torch.Size([32, 10, 3000])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of

0

In [ ]:
from eegproject.models.cnn_classifier import CNNClassifier
from eegproject.models.lstm_classifier import LSTMClassifier
from eegproject.models.cnn_encoder import CNNEncoder
from torch.nn.utils.rnn import pad_sequence
import torch

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 15
sequence_length = 35

In [ ]:
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score

def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model.to(device).train()
    cum_loss = 0
    n_objects = 0
    state = None
    for features, y, sl in tqdm(iterate_batch_multiple_seq_minibatches(train_dataset_.X, train_dataset_.y, batch_size, sequence_length)):
        if sl:
            state = model.init_state(device)
        
        features = features.to(device)
        y = y.to(device)
                
        preds, new_state = model(features, state)

        loss = criterion(preds[y != -1], y[y != -1])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        state = new_state
        
        cur_obj = torch.sum(y != -1).cpu()
        cum_loss += loss.detach().cpu().numpy() * cur_obj.numpy()
        n_objects += cur_obj
        
    return cum_loss / n_objects


def predict(model, test_dataloder, criterion, device="cuda:0"):
    model.to(device).eval()
    with torch.no_grad():        
        predicts = torch.tensor([])
        true_values = torch.tensor([])
        cum_loss = 0
        n_objects = 0
        state = None
        for features, y, sl in tqdm(iterate_batch_multiple_seq_minibatches(test_dataset_.X, test_dataset_.y, batch_size, sequence_length)):
            if sl:
                state = model.init_state(device)
            
            features = features.to(device)
            y = y.to(device)
            
            cur, state = model(features, state)
            cur = cur[y != -1]
            
            predicts = torch.cat([predicts, torch.argmax(cur.cpu(), axis=1)])
            true_values = torch.cat([true_values, y[y != -1].cpu()])

            cur_obj = torch.sum(y != -1).cpu()
            n_objects += cur_obj
            cum_loss += criterion(cur, y[y != -1]).cpu().item() * cur_obj
        
        return cum_loss / n_objects, predicts, true_values
    

def train(model, train_dataloader, test_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):
        print('Train')
        train_loss = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        print('Evaluate')
        val_loss, predicted, true = predict(model, test_dataloader, criterion, device)
        if scheduler is not None:
            scheduler.step(val_loss)
            
        accuracy = accuracy_score(predicted, true)
        print('Epoch {}, val loss {:.3f}, train loss {:.3f}, accuracy {:.3f}'\
              .format(epoch + 1, val_loss, train_loss, accuracy))

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [ ]:
encoder = torch.load('cnn_encoder.pt', map_location='cpu')
model = LSTMClassifier(encoder, batch_size=batch_size, bidirectional=True, hidden_size=32)

clip_value = 1.0
for p in model.parameters():
    p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5, verbose=True)

In [ ]:
val_loss, predicted, true = predict(model, None, criterion, device)
accuracy_score(predicted, true)

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of

0.36182561146534536

In [ ]:
train(model, None, None, criterion, optimizer, device, 35, scheduler)

Train


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of

Evaluate


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of

Evaluate


0it [00:00, ?it/s]

Epoch 35, val loss 0.980, train loss 0.551, accuracy 0.648


In [ ]:
torch.save(model, 'lstm_classifier.pt')